In [459]:
import pandas as pd
from pymongo import MongoClient
import warnings
import re
from fuzzywuzzy import fuzz
import numpy as np
import json

#### Loading Data

In [461]:
client = MongoClient()
client.list_database_names()

['admin',
 'books',
 'catalog',
 'config',
 'events',
 'legistlation',
 'local',
 'resumes']

In [463]:
resumes = client.resumes.resumes
resumes.count()

29634

In [5]:
df = pd.DataFrame(list(resumes.find({'Title':{'$exists': 1}},
                                    {'Title': 1, 'Companies': 1, 'Resume Summary': 1, 'Current Location': 1, 
                                     'Start Dates': 1, 'Work Experiences': 1, 'Universities': 1, 'Degrees': 1,
                                     'in': 1, 'Skills': 1, 'Additionals': 1, 'Link': 1, 'Whole Resume': 1,
                                     '_id': 0})))

In [6]:
df.head()

,Additionals,Companies,Current Location,Degrees,Link,Resume Summary,Skills,Start Dates,Title,Universities,Whole Resume,Work Experiences,in
0,[Additional InformationSkills• Financial repor...,"[Yoox Net-A-Porter Group, Yoox Net-A-Porter Gr...",[],[Bachelor's],/resume/0002cdce16f56ac0?s=l%3D%26q%3DData%252...,[Goal-oriented team player with experience in ...,"[Business Administration and Marketing, CUSTOM...",[May 2012],[Financial Data Analyst],[Bloomsburg University of Pennsylvania],[Financial Data AnalystFinancial Data Analyst ...,[• Formulate sales reports and other metrics u...,[Business Administration and Marketing]
1,[Additional InformationSkillsTechnical: Profic...,"[Thimble LLC/DHS contract, Florida State Unive...","[Washington, DC]",[B.S. in Statistics],/resume/000407ced092c381?s=l%3D%26q%3DData%252...,"[Detail-oriented, methodical, and analytical, ...","[Statistics, Microsoft Office, 5 years, SAS, 2...",[May 2016],[Data Analyst],[Florida State University],"[Data AnalystWashington, DCDetail-oriented, me...",[Manually adjusted and corrected data collecti...,[Statistics]
2,[Additional InformationTechnical Skills- Windo...,"[New York State Office of Mental Health, New Y...","[New York, NY]","[Master of Science, Bachelor of Engineering]",/resume/000488d3ffec9813?s=l%3D%26q%3DData%252...,[Over 15 years of experience in information te...,"[Information Systems, Mechanical and Productio...",[January 2000],[Business Intelligence/Data Warehouse Architect],"[Arizona State University, Annamalai University]",[Business Intelligence/Data Warehouse Architec...,[- Worked on Data Warehouse projects from ince...,"[Information Systems, Mechanical and Production]"
3,[Additional InformationTechnical Skills- Windo...,"[New York State Office of Mental Health, New Y...","[New York, NY]","[Master of Science, Bachelor of Engineering]",/resume/000488d3ffec9813?s=l%3D%26q%3DData%252...,[Over 15 years of experience in information te...,"[Information Systems, Mechanical and Productio...",[January 2000],[Business Intelligence/Data Warehouse Architect],"[Arizona State University, Annamalai University]",[Business Intelligence/Data Warehouse Architec...,[- Worked on Data Warehouse projects from ince...,"[Information Systems, Mechanical and Production]"
4,[Certifications/LicensesLooker Data Analyst & ...,"[Big Squid, Python, Marketware, PwC, The Unive...","[Salt Lake City, UT]","[M.S., BS]",/resume/0006d4f5109dfb54?s=l%3D%26q%3DData%252...,"[As the Lead Data Consultant at Big Squid, I s...","[Information Systems, Accounting, Snowflake, L...",[August 2012],[Lead Data Consultant],"[The University of Utah, BYU-Idaho]",[Lead Data ConsultantData Analytics Consultant...,[- Built our data warehousing and data enginee...,"[Information Systems, Accounting]"


#### One Big Cleaning Function

In [475]:
def cleaning(df):
    
    # columns
    df_ = df[['Additionals', 'Companies', 'Current Location', 'Degrees',
       'Resume Summary', 'Skills', 'Start Dates', 'Title', 'Universities',
       'Whole Resume', 'Work Experiences', 'in']]
    
    # append skills and get rid of duplicate scrapes in the 'in' column
    only_skillz_list = []
    for skillz, degreez in zip(df_['Skills'], df_['in']):
        only_skillz_list.append([skill for skill in skillz if skill not in degreez])
    df_['Skills'] = only_skillz_list
    
    # remove brackets and ''
    for column in df_.columns:
        df_[column] = [element[1:-1] for element in df_[column].astype(str)]
    
    # remove duplicates and reset index
    df_.drop_duplicates(keep='first',inplace=True)
    df_.reset_index(drop=True,inplace=True)
    
    # only include rows where there are no error fields
    df_ = df_[df_['Additionals'] != "'error'"]
    
    # remove text with weird symbols as this could be picked up in our W2V model
    per_column = []
    for column in df_.columns:
        column_lis = []
        for element in df_[column]:
            text = []
            element = element.replace("Additional Information","").replace("Work Experience",
                                "").replace("Certifications","").replace("'error'","").replace("SKILLS","")
            for words in element.split():
                words = words.replace("●","").replace("•","").replace("◆","").replace("❖","").replace("✓",
                        "").replace("★","").replace("☆","").replace("➢","")
                text.extend(re.findall('[A-Z]*[^A-Z]+', words))
            text = " ".join(text)
            column_lis.append(text)
        per_column.append(column_lis)

    for column, i in zip(df_.columns, range(len(df_.columns))):
        df_[column] = per_column[i]
    
    # calculate years of work experience based on start date
    starts = []
    for date in df_['Start Dates']:
        try:
            starts.append(2018 - eval(date.split()[2][:4]))
        except:
            starts.append("")
    df_['Start Dates'] = starts
    
    # classifying job titles according to these 3 roles (y-dependent variables)
    roles = []
    for role in df_['Title']:
        if (fuzz.partial_ratio(role, 'Scientist') == 100) or (fuzz.partial_ratio(role, 'Science') == 100)\
         or (fuzz.partial_ratio(role, 'Machine Learning') == 100):
            roles.append('Scientist')
        elif (fuzz.partial_ratio(role, 'Engineer') == 100) or (fuzz.partial_ratio(role, 'Warehous') == 100)\
         or (fuzz.partial_ratio(role, 'Architect') == 100) or (fuzz.partial_ratio(role, 'Big Data') == 100)\
            or (fuzz.partial_ratio(role, 'Developer') == 100):
            roles.append('Engineer')
        elif (fuzz.partial_ratio(role, 'Analyst') == 100) or (fuzz.partial_ratio(role, 'Analytics') == 100)\
         or (fuzz.partial_ratio(role, 'Associate') == 100) or (fuzz.partial_ratio(role, 'Intelligen') == 100):
            roles.append('Analyst')
        elif (fuzz.partial_ratio(role, 'Intern') == 100):
            roles.append('None')
        else:
            roles.append('None')
    df_['y'] = roles
    df_usable = df_[df_['y'] != 'None']
    df_usable = pd.concat([df_usable, pd.get_dummies(df_usable['y'])], axis=1)
    
    # label items that were additionally scraped
    what = []
    for additional in df_usable['Additionals']:
        if (fuzz.partial_ratio(additional, 'Additional Information') == 100):
            what.append('Additionals')
        elif (fuzz.partial_ratio(additional, 'Skills') == 100):
            what.append('Skills')
        elif (fuzz.partial_ratio(additional, 'Education') == 100):
            what.append('Education')
        elif (fuzz.partial_ratio(additional, 'Links') == 100):
            what.append('Links')
        elif (fuzz.partial_ratio(additional, 'Work Experience') == 100):
            what.append('Exp')
        else:
            what.append('Other')
    df_usable['What'] = what
    
    # remove duplicates, leave blank
    for item in ['Skills','Education','Links','Exp']:
        mask = df_usable[df_usable['What'] == item]
        mask[['Additionals']] = " "
        df_usable[df_usable['What'] == item] = mask
    
    # identify types of degrees - bachelors, masters, PhDs and number
    degrees = []
    for degree in df_usable['Degrees']:
        degree = degree.split(", ")
        d = []
        for deg in degree:
            try:
                d.append(deg[2])
            except:
                d.append(deg)
        degrees.append(d)

    df_usable['Degs'] = degrees
    df_usable['Degs'] = df_usable['Degs'].astype(str)
    df_usable['Degs'] = df_usable['Degs'].str.strip("[]")
    B = []
    M = []
    P = []
    
    for degree in df_usable['Degs']:
        bt = 0
        mt = 0
        pt = 0
        for deg in degree:
            if 'B' in deg:
                bt += 1
            if 'M' in deg:
                mt += 1
            if 'P' in deg:
                pt += 1
        B.append(bt)
        M.append(mt)
        P.append(pt)
    df_usable['Bachelors'] = B
    df_usable['Masters'] = M
    df_usable['PhD'] = P

    warnings.filterwarnings('ignore')
    
    return df_usable

df_usable = cleaning(df)

#### Store cleaned data in MongoDB.

In [477]:
client = MongoClient('localhost', 27017)
db = client['resumes']

df_clean = json.loads(df_usable.T.to_json()).values()
db.resumes_cleaned.insert(df_clean)

[ObjectId('5b981a629bd8d25761955744'),
 ObjectId('5b981a629bd8d25761955745'),
 ObjectId('5b981a629bd8d25761955746'),
 ObjectId('5b981a629bd8d25761955747'),
 ObjectId('5b981a629bd8d25761955748'),
 ObjectId('5b981a629bd8d25761955749'),
 ObjectId('5b981a629bd8d2576195574a'),
 ObjectId('5b981a629bd8d2576195574b'),
 ObjectId('5b981a629bd8d2576195574c'),
 ObjectId('5b981a629bd8d2576195574d'),
 ObjectId('5b981a629bd8d2576195574e'),
 ObjectId('5b981a629bd8d2576195574f'),
 ObjectId('5b981a629bd8d25761955750'),
 ObjectId('5b981a629bd8d25761955751'),
 ObjectId('5b981a629bd8d25761955752'),
 ObjectId('5b981a629bd8d25761955753'),
 ObjectId('5b981a629bd8d25761955754'),
 ObjectId('5b981a629bd8d25761955755'),
 ObjectId('5b981a629bd8d25761955756'),
 ObjectId('5b981a629bd8d25761955757'),
 ObjectId('5b981a629bd8d25761955758'),
 ObjectId('5b981a629bd8d25761955759'),
 ObjectId('5b981a629bd8d2576195575a'),
 ObjectId('5b981a629bd8d2576195575b'),
 ObjectId('5b981a629bd8d2576195575c'),
 ObjectId('5b981a629bd8d2

In [478]:
df_usable.head()

,Additionals,Companies,Current Location,Degrees,Resume Summary,Skills,Start Dates,Title,Universities,Whole Resume,...,in,y,Analyst,Engineer,Scientist,What,Degs,Bachelors,Masters,PhD
0,,"' Yoox Net- A- Porter Group', ' Yoox Net- A- P...",'',""" Bachelor's""",' Goal-oriented team player with experience in...,"' SERVICE', '3 years', ' SALES', '3 years', ' ...",6,' Financial Data Analyst',' Bloomsburg University of Pennsylvania',""" Financial Data Analyst Financial Data Analys...",...,' Business Administration and Marketing',Analyst,1,0,0,Skills,'B',1,0,0
1,,"' Thimble LLC/ contract', ' Florida State Univ...","' Washington, DC'",' B. S. in Statistics',"' Detail-oriented, methodical, and analytical,...","' Microsoft Office', '5 years', ' SAS', '2 yea...",2,' Data Analyst',' Florida State University',"' Data Analyst Washington, DCDetail-oriented, ...",...,' Statistics',Analyst,1,0,0,Skills,'B',1,0,0
2,,"' New York State Office of Mental Health', ' N...","' New York, NY'","' Master of Science', ' Bachelor of Engineering'",' Over 15 years of experience in information t...,"' ORACLE', '10+ years', ' PL/ SQL', '10+ years...",18,' Business Intelligence/ Data Warehouse Archit...,"' Arizona State University', ' Annamalai Unive...",""" Business Intelligence/ Data Warehouse Archit...",...,"' Information Systems', ' Mechanical and Produ...",Engineer,0,1,0,Skills,"'M', 'B'",1,1,0
4,,"' Allied Benefit Systems', ' Expedia, Inc.', '...","' Chicago, IL'",' B. Sc.',,"' VBA', '3 years', ' Excel', '3 years', ' Powe...",5,' Data Analyst',' Statistics B. Sc. Concordia University Montr...,"' Data Analyst Data Analyst Bellevue, WAData A...",...,' Mathematics of Finance',Analyst,1,0,0,Skills,'B',1,0,0
5,'/ Licenses VMware Certified Professional ( VC...,"' Presidio', ' Presidio', ' Global', ' The Geo...","' Reston, VA'","' Master of Science', ' Bachelor of Engineering'",""" 6+ years of experience in Data Center design...","' Vmware', '6 years', ' Cisco', '6 years', ' E...",8,' Sr. Data Center Engineer',"' The George Washington University', ' Univers...",""" Sr. Data Center Engineer Sr. Data Center Eng...",...,"' Electrical Engineering', ' Electronics Engin...",Engineer,0,1,0,Other,"'M', 'B'",1,1,0


In [473]:
df_usable.columns

Index(['Additionals', 'Companies', 'Current Location', 'Degrees',
       'Resume Summary', 'Skills', 'Start Dates', 'Title', 'Universities',
       'Whole Resume', 'Work Experiences', 'in', 'y', 'Analyst', 'Engineer',
       'Scientist', 'What', 'Degs', 'Bachelors', 'Masters', 'PhD'],
      dtype='object')

In [450]:
pd.concat([df_usable, pd.get_dummies(df_usable['y'])], axis=1)

,Additionals,Companies,Current Location,Degrees,Resume Summary,Skills,Start Dates,Title,Universities,Whole Resume,...,in,y,What,Degs,Bachelors,Masters,PhD,Analyst,Engineer,Scientist
0,' Additional Information Skills Financial repo...,"' Yoox Net- A- Porter Group', ' Yoox Net- A- P...",'',""" Bachelor's""",' Goal-oriented team player with experience in...,"' SERVICE', '3 years', ' SALES', '3 years', ' ...",6,' Financial Data Analyst',' Bloomsburg University of Pennsylvania',""" Financial Data Analyst Financial Data Analys...",...,' Business Administration and Marketing',Analyst,Additionals,'B',1,0,0,1,0,0
1,' Additional Information Skills Technical: Pro...,"' Thimble LLC/ contract', ' Florida State Univ...","' Washington, DC'",' B. S. in Statistics',"' Detail-oriented, methodical, and analytical,...","' Microsoft Office', '5 years', ' SAS', '2 yea...",2,' Data Analyst',' Florida State University',"' Data Analyst Washington, DCDetail-oriented, ...",...,' Statistics',Analyst,Additionals,'B',1,0,0,1,0,0
2,' Additional Information Technical Skills- Win...,"' New York State Office of Mental Health', ' N...","' New York, NY'","' Master of Science', ' Bachelor of Engineering'",' Over 15 years of experience in information t...,"' ORACLE', '10+ years', ' PL/ SQL', '10+ years...",18,' Business Intelligence/ Data Warehouse Archit...,"' Arizona State University', ' Annamalai Unive...",""" Business Intelligence/ Data Warehouse Archit...",...,"' Information Systems', ' Mechanical and Produ...",Engineer,Additionals,"'M', 'B'",1,1,0,0,1,0
4,NaN,"' Allied Benefit Systems', ' Expedia, Inc.', '...","' Chicago, IL'",' B. Sc.','error',"' VBA', '3 years', ' Excel', '3 years', ' Powe...",5,' Data Analyst',' Statistics B. Sc. Concordia University Montr...,"' Data Analyst Data Analyst Bellevue, WAWork E...",...,' Mathematics of Finance',Analyst,Skills,'B',1,0,0,1,0,0
5,'/ Licenses VMware Certified Professional ( VC...,"' Presidio', ' Presidio', ' Global', ' The Geo...","' Reston, VA'","' Master of Science', ' Bachelor of Engineering'",""" 6+ years of experience in Data Center design...","' Vmware', '6 years', ' Cisco', '6 years', ' E...",8,' Sr. Data Center Engineer',"' The George Washington University', ' Univers...",""" Sr. Data Center Engineer Sr. Data Center Eng...",...,"' Electrical Engineering', ' Electronics Engin...",Engineer,Other,"'M', 'B'",1,1,0,0,1,0
6,NaN,"' Pacific Pulmonary Services', ' Pacific Pulmo...","' Bakersfield, CA'",' Pursuing Associates Degree',' Lead Data Analyst with experience interpreti...,"' Microsoft Excel', '3 years', ' Excel', ' Les...",12,' Lead Data Analyst',"' Bakersfield College', ' Ridgeview High School'","' Lead Data Analyst Bakersfield, CALead Data A...",...,,Analyst,Skills,'P',0,0,1,1,0,0
7,' Additional Information Dedicated professiona...,"' Duke- Energy', ' Kooks Custom Headers, Inc',...","' Charlotte, NC'",' Diploma','error',"' BAAN', '10+ years', ' GROUP', '6 years', ' P...",30,"' Supply Chain Data Analyst, Contractor'",' College of Technology',"' Supply Chain Data Analyst, Contractor Supply...",...,' Business Computer Applications',Analyst,Additionals,'D',0,0,0,1,0,0
8,' Additional Information SKILLSComputer Skills...,"' Willis Towers Watson', ' Design Corp', ' Fid...",'','',' Proven financial data analyst with experienc...,"' Actuarial Science', ' Actuarial Science', ' ...",1,"' Analyst, Global Data Services'","' Johns Hopkins University', ' Boston Universi...",""" Analyst, Global Data Services Analyst, Globa...",...,,Analyst,Additionals,"""''""",0,0,0,1,0,0
9,' Additional Information Highlights of Qualifi...,' Sunshine Health Plan & Superior Health Plan'...,'','','error',"' SQL', '6 years', ' CLAIMS', '3 years', ' MIC...",7,' Data Analyst II',"' Naval Nuclear Power School', ' Mc Gregor Hig...",""" Data Analyst IIData Analyst - Sunshine Healt...",...,,Analyst,Additionals,"""''""",0,0,0,1,0,0
11,""" Additional Information SKILLSI'm an eight ye...","' JPMC', ' Marriott ( Merge IT)', ' Virtu Stre...","' Ashbu